### Project Overview:
- For this project you will need to fetch Adj Close every company in the Russell 2000 for 2020 (Time Frame Jan 1 - current) 
- We would like to see the Dips and Gains/ Daily Returns of every stock under priced under 10 dollars in ascending order.
- Repeat the same process for every company in the entire index
- Using portfolio optimization techniques find 20 companies in a portfolio that would generate a "Good" return during the current Pandemic. 
- We want you to than go back 5 years and see how these companies performed pre-Covid. What is the max return we would have received during this 5 year period considering volatility.

### Project Approach
- Fetch all companies in Russell 2000
- Use pandas Datareader to get the Adj Close
- Calculate the Daily returns for each security
- Find the top 20 Gains and Dips for returns for the current day
- Put every security into a portfolio and optimize the portfolio with 5 years of data (2015-2019 end)
- Find the top companies the make up the index by weight and isolate them into a portfolio of their own
- Run a portfolio optimization on the portfolio using Markowitz Efficient Frontier and see what the ortfolio would have returned pre-Covid. 

### Acknowledgements
- Data was downloaded provided from Ben Reynolds at Secure Dividends
- https://www.suredividend.com/

### libraries


In [228]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from pandas_datareader import data as web

In [229]:
russ = pd.read_excel("resources/Russle_2000_2020.xlsx", index_col='Ticker')

### Data Overview 
- Data below provides all of the data tickers and companies in the Russell 2000

In [230]:
russ

,Name
Ticker,
AAN,"Aaron's, Inc."
AAOI,"Applied Optoelectronics, Inc."
AAON,"AAON, Inc."
AAT,"American Assets Trust, Inc."
AAWW,"Atlas Air Worldwide Holdings, Inc."
...,...
ZIXI,Zix Corp.
ZUMZ,"Zumiez, Inc."
ZUO,"Zuora, Inc."


In [231]:
russ.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1999 entries, AAN to ZYXI
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    1999 non-null   object
dtypes: object(1)
memory usage: 31.2+ KB


### Adj Closes
- Will import data already fetched previously
- Will download the Adj Close for each security for 2020 beginning 2020-1-1 to current using pandas Datareader

In [232]:
tickers = list(russ.index)

In [233]:
len(tickers)

1999

In [234]:
data = pd.DataFrame()

In [235]:
#failed = []
#passed = []
#for x in tickers:
   # try:
      #  data[x] = web.DataReader(x, data_source= "yahoo", start = "2020-6-1")["Adj Close"]
      #  passed.append(x)
   # except (IOError, KeyError):
        #msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
       # failed.append(x)

### Passed Tickers
- There are 1941 tickers that passed in the Yahoo data reader library
- We will use these for our analysis

### Saving Passed Tickers

In [236]:
penny = pd.read_csv("resources/current/penny_tickers.csv", index_col= "Unnamed: 0")

In [237]:
penny

,under 10
0,ABEO
1,ACER
2,ACOR
3,ACRS
4,ACRX
...,...
341,YCBD
342,YRCW
343,ZIOP
344,ZIXI


In [238]:
penny_tickers = list(penny["under 10"])

In [239]:
penny_data = pd.DataFrame()

In [240]:
failed = []
passed = []
for x in list(penny_tickers):
    try:
        penny_data[x] = web.DataReader(x, data_source= "yahoo", start = "2020-8-10")["Adj Close"]
        passed.append(x)
    except (IOError, KeyError):
        msg = 'Failed to read symbol: {0!r}, replacing with NaN.'
        failed.append(x)

In [241]:
penny_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2020-08-10 to 2020-08-14
Columns: 340 entries, ABEO to ZYNE
dtypes: float64(340)
memory usage: 13.3 KB


In [242]:
#pd.DataFrame(penny_tickers, columns=["under 10"]).to_csv("resources/current/penny_tickers.csv")

## Checking the Daily returns for  the Russell 2000
- Will check today's Date for the Dips and Spikes in the entire index
- Will find the to 20 companies that had a negative return for todays date

In [243]:
todays_Date = penny_data.iloc[-1:].iloc[0].name

# LOOK HERE FOR Gains

In [244]:
prices = penny_data.iloc[-1:].transpose()

In [245]:
sorted_gains= penny_data.pct_change().iloc[-1:].transpose().sort_values(todays_Date, ascending = False)

In [246]:
gains_andPrice = pd.concat([sorted_gains, prices], axis=1).dropna()

In [247]:
gains_andPrice.columns = ["gains", "price"]

In [248]:
gains_andPrice.head(10)

,gains,price
OCN,13.879433,20.9800
IPI,9.730666,12.8768
PRTY,0.275862,2.5900
XERS,0.136240,4.1700
LXU,0.131443,2.1950
BDSI,0.126652,5.1150
CRK,0.105482,6.6550
MR,0.101490,5.9150
SNCR,0.092417,4.6100
SD,0.090361,1.8100


## Dips 

In [ ]:
top_20_dips = data.pct_change()[-1:].transpose()

In [ ]:
top_20_dips.plot(kind = "bar", figsize = (16,6))

## Spikes

In [ ]:
top_20_Spikes = data.pct_change()[-1:].transpose().sort_values("2020-07-13", ascending = False).dropna().head(20)

In [ ]:
top_20_Spikes

In [ ]:
top_20_Spikes.plot(kind = "bar", figsize = (16,6))

### Optimizing the Russell 2000 as a portfolio
- Will put every company in a portfolio and see which companies would have held the mose weight during the 2020 covid 19 crisis thu far. 
- Will use markowitz Portfolio theor to optimize the Russell
- Will use 2000 randomly allocated portfolios to get the efficient frontier using a combination of volatility and expected returns
- We will drop companies with missing data for the time being

In [ ]:
russell_Returns = data.dropna(axis=1).pct_change()

In [ ]:
tic = list(russell_Returns.columns)

In [ ]:
n_portfolios = 2000
all_weights = np.zeros((n_portfolios, len(tic)))
all_returns = np.zeros(n_portfolios)
all_vol = np.zeros(n_portfolios)
all_sharp = np.zeros(n_portfolios)

for ind in range(n_portfolios):
    weights = np.array(np.random.random(len(tic)))
    weights = weights/weights.sum()   
    all_weights[ind,:] = weights
    
    all_returns[ind] = np.sum(russell_Returns.mean() * weights) * 252
    all_vol[ind] = np.dot(weights.T, np.dot(russell_Returns.cov() * 252, weights))
    all_sharp[ind] = all_returns[ind]/ all_vol[ind]

### Plotting the Frontier
- The frontier will provide a good insight on the max return hightest sharp and the lowest volatility of the portfolios

In [ ]:
hightest_return = all_returns.argmax()

In [ ]:
lowest_vol = all_vol.argmin()

In [ ]:
higherst_sharp = all_sharp.argmax()

In [ ]:
all_returns.max()

#### Frontier 
- We see two things in the froniter
- The portfolio with the highest sharp is provides the same return as the portfolio with the highrest return 
- Appears the highest return for a portfolio containing every company in the Russell is in the negative
- This is not realistic and the main purpose was to find the top 20 weighted companies in 2020
- We will fetch these below

In [ ]:
plt.figure(figsize = (16,10))
plt.scatter(all_vol, all_returns, c=all_sharp)
plt.scatter(all_vol[hightest_return], all_returns[hightest_return], c="r", s = 160)
plt.scatter(all_vol[lowest_vol], all_returns[lowest_vol], c="orange", s = 80)
plt.scatter(all_vol[higherst_sharp], all_returns[higherst_sharp], c="black", s = 60)

plt.colorbar(label = "Sharp")

## Bringing the data together 
- Creating a data frame for the weights returns and sharp

In [ ]:
frontier_df = pd.DataFrame(all_returns, columns=["returns"])

In [ ]:
frontier_df = pd.concat([frontier_df, pd.DataFrame(all_vol, columns=["vol"])], axis=1)

In [ ]:
shp_df = pd.DataFrame(all_sharp, columns=["shp"])

In [ ]:
frontier_df = pd.concat([frontier_df, shp_df], axis=1)

In [ ]:
weights_df = pd.DataFrame(all_weights, columns=tic)

In [ ]:
frontier_df = pd.concat([frontier_df, weights_df], axis=1)

In [ ]:
## frontier_df.to_csv("resources/current/frontier_entier_russell.csv")

### Lets find the portfolio with the highest returns
- Will take the top 20 compaines by weight in this portfolio and create a new portfolio with just these 20 companies

In [ ]:
highest_ret_port = pd.DataFrame(frontier_df.sort_values("returns", ascending = False).loc[1869])

In [ ]:
highest_ret_port.drop(["returns", "vol", "shp"], inplace=True)

## Below are the top 20 weighted companies 
- will create a portfolio for just these companies and see what our returns would have been for 2020

In [ ]:
top_20_weighted = highest_ret_port.sort_values(1869, ascending = False).head(20)

In [ ]:
top20_tickers = list(top_20_weighted.index)

In [ ]:
top_20_df = pd.DataFrame()

In [ ]:
for y in top20_tickers:
    top_20_df[y] = web.DataReader(y, data_source="yahoo", start = "2020-1-1")["Adj Close"]

In [ ]:
top_20_df.plot(figsize = (16,6))

In [ ]:
top_20_df

In [ ]:
top_20_returns = top_20_df.pct_change()

### Volatility 
- This portfolio has a high vol minly occurring with 2 companies
- Lets calculate the volatility below 

In [ ]:
np.sum(top_20_returns.mean() * 252)

In [ ]:
top_20_returns.plot(figsize = (16,6), legend = False)

In [ ]:
n_portfolios = 2000
all_weights = np.zeros((n_portfolios, len(top_20_df.columns)))
all_returns = np.zeros(n_portfolios)
all_vol = np.zeros(n_portfolios)
all_sharp = np.zeros(n_portfolios)

for ind in range(n_portfolios):
    weights = np.array(np.random.random(len(top_20_df.columns)))
    weights = weights/weights.sum()   
    all_weights[ind,:] = weights
    
    all_returns[ind] = np.sum(top_20_returns.mean() * weights) * 252
    all_vol[ind] = np.dot(weights.T, np.dot(top_20_returns.cov() * 252, weights))
    all_sharp[ind] = all_returns[ind]/ all_vol[ind]

In [ ]:
max_return = all_returns.argmax()

In [ ]:
max_Sharp = all_sharp.argmax()

In [ ]:
lowest_vol = all_vol.argmin()

### Summary
- This portfolio would have yielded you 38% return year to date 
- Not bad even during Covid 

In [ ]:
all_returns.max()

In [ ]:
all_sharp.max()

In [ ]:
all_vol.min()

In [ ]:
plt.figure(figsize=(16,6))
plt.scatter(all_vol, all_returns, c=all_sharp)
plt.scatter(all_vol[max_return], all_returns[max_return], c="r", s = 80)
plt.scatter(all_vol[max_Sharp], all_returns[max_Sharp], c="black", s = 40)
plt.scatter(all_vol[lowest_vol], all_returns[lowest_vol], c="r", s = 40)
plt.colorbar(label = "Sharp")